In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, False, 15)

In [12]:

if file_parameters['watershed'] == 'nak':
    run_num = range(len(folder))
else:
    run_num = [0]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 195ms/step - gen_loss: 174.8491 - disc_loss: 0.7364 - rmse: 1.3369 - val_loss: 0.9348
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 104.1296 - disc_loss: 0.5436 - rmse: 0.9959 - val_loss: 0.7837
Epoch 3/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 80.1514 - disc_loss: 0.4354 - rmse: 0.8354 - val_loss: 0.8224
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 74.8250 - disc_loss: 0.4072 - rmse: 0.8309 - val_loss: 0.7099
Epoch 5/2000
1/1 [==============================] - 0s 2

Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1006 - disc_loss: 0.3514 - rmse: 0.2892 - val_loss: 0.2661
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7378 - disc_loss: 0.3741 - rmse: 0.3634 - val_loss: 0.3222
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7946 - disc_loss: 0.3712 - rmse: 0.3130 - val_loss: 0.2915
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2169 - disc_loss: 0.3517 - rmse: 0.2917 - val_loss: 0.3305
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 39.8953 - disc_loss: 0.3563 - rmse: 0.5461 - val_loss: 0.3393
Epoch 62/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6869 - disc_loss: 0.3493 - rmse: 0.2826 - val_loss: 0.3213
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6640 - disc_loss: 0.3455 - rmse: 0.3188 - val_loss: 0.2849
Epoch 64/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0486 - disc_loss: 0.2864 - rmse: 0.2670 - val_loss: 0.2628
Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 31.5488 - disc_loss: 0.3020 - rmse: 0.4749 - val_loss: 0.3415
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.7525 - disc_loss: 0.2778 - rmse: 0.3341 - val_loss: 0.3093
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0772 - disc_loss: 0.2879 - rmse: 0.2936 - val_loss: 0.4470
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 51.0049 - disc_loss: 0.2943 - rmse: 0.5867 - val_loss: 0.3054
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4929 - disc_loss: 0.2979 - rmse: 0.2822 - val_loss: 0.2975
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 38.2745 - disc_loss: 0.2767 - rmse: 0.5994 - val_loss: 0.3432
Epoch 122/2000
1/1 [============

1/1 [==============================] - 0s 21ms/step - gen_loss: 4.9156 - disc_loss: 0.2762 - rmse: 0.2845 - val_loss: 0.3750
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1347 - disc_loss: 0.2512 - rmse: 0.2294 - val_loss: 0.2310
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6277 - disc_loss: 0.2534 - rmse: 0.2394 - val_loss: 0.2570
Epoch 176/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 28.4012 - disc_loss: 0.2605 - rmse: 0.5056 - val_loss: 0.2462
Epoch 177/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5386 - disc_loss: 0.2700 - rmse: 0.2545 - val_loss: 0.2549
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3686 - disc_loss: 0.2363 - rmse: 0.2290 - val_loss: 0.2461
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2552 - disc_loss: 0.2692 - rmse: 0.2420 - val_loss: 0.2652
Epoch 180/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0917 - disc_loss: 0.2501 - rmse: 0.2616 - val_loss: 0.3239
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5208 - disc_loss: 0.2375 - rmse: 0.3103 - val_loss: 0.2862
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5190 - disc_loss: 0.2575 - rmse: 0.2794 - val_loss: 0.2654
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5335 - disc_loss: 0.2462 - rmse: 0.2693 - val_loss: 0.3442
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2146 - disc_loss: 0.2400 - rmse: 0.2643 - val_loss: 0.2799
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8792 - disc_loss: 0.2210 - rmse: 0.2466 - val_loss: 0.2626
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6182 - disc_loss: 0.2162 - rmse: 0.2829 - val_loss: 0.3869
Epoch 238/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9801 - disc_loss: 0.1640 - rmse: 0.2068 - val_loss: 0.2317
Epoch 1740/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8504 - disc_loss: 0.1591 - rmse: 0.2184 - val_loss: 0.2065
Epoch 1741/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3833 - disc_loss: 0.1659 - rmse: 0.2360 - val_loss: 0.1861
Epoch 1742/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5426 - disc_loss: 0.1603 - rmse: 0.1895 - val_loss: 0.2016
Epoch 1743/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2269 - disc_loss: 0.1536 - rmse: 0.1792 - val_loss: 0.2559
Epoch 1744/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1565 - disc_loss: 0.1499 - rmse: 0.2263 - val_loss: 0.1996
Epoch 1745/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4004 - disc_loss: 0.1591 - rmse: 0.2166 - val_loss: 0.2113
Epoch 1746/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4676 - disc_loss: 0.1548 - rmse: 0.1917 - val_loss: 0.2059
Epoch 1798/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5976 - disc_loss: 0.1488 - rmse: 0.1616 - val_loss: 0.2522
Epoch 1799/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6312 - disc_loss: 0.1514 - rmse: 0.2165 - val_loss: 0.2102
Epoch 1800/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5504 - disc_loss: 0.1543 - rmse: 0.1701 - val_loss: 0.1956
Epoch 1801/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.8063 - disc_loss: 0.1530 - rmse: 0.6005 - val_loss: 0.1739
Epoch 1802/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4718 - disc_loss: 0.1591 - rmse: 0.2011 - val_loss: 0.1979
Epoch 1803/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2672 - disc_loss: 0.1585 - rmse: 0.1842 - val_loss: 0.2162
Epoch 1804/2000
1/1 [=======

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9247 - disc_loss: 0.1568 - rmse: 0.2277 - val_loss: 0.1940
Epoch 1856/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5489 - disc_loss: 0.1563 - rmse: 0.2190 - val_loss: 0.2021
Epoch 1857/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.7429 - disc_loss: 0.1585 - rmse: 0.6458 - val_loss: 0.6517
Epoch 1858/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0321 - disc_loss: 0.1528 - rmse: 0.2138 - val_loss: 0.1830
Epoch 1859/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9292 - disc_loss: 0.1583 - rmse: 0.1898 - val_loss: 0.1884
Epoch 1860/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5985 - disc_loss: 0.1514 - rmse: 0.1835 - val_loss: 0.6661
Epoch 1861/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 24.8278 - disc_loss: 0.1594 - rmse: 0.5185 - val_loss: 0.2263
Epoch 1862/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7220 - disc_loss: 0.1664 - rmse: 0.1935 - val_loss: 0.1924
Epoch 1914/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2320 - disc_loss: 0.1541 - rmse: 0.2076 - val_loss: 0.1896
Epoch 1915/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.3497 - disc_loss: 0.1510 - rmse: 0.7009 - val_loss: 0.6537
Epoch 1916/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5412 - disc_loss: 0.1602 - rmse: 0.2058 - val_loss: 0.1854
Epoch 1917/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1499 - disc_loss: 0.1494 - rmse: 0.2289 - val_loss: 0.2070
Epoch 1918/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3494 - disc_loss: 0.1512 - rmse: 0.1826 - val_loss: 0.2289
Epoch 1919/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.5844 - disc_loss: 0.1464 - rmse: 0.7002 - val_loss: 0.1957
Epoch 1920/2000
1/1 [======

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4224 - disc_loss: 0.1503 - rmse: 0.1741 - val_loss: 0.4533
Epoch 1972/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.5880 - disc_loss: 0.1594 - rmse: 0.5855 - val_loss: 0.1892
Epoch 1973/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.8831 - disc_loss: 0.1470 - rmse: 0.2225 - val_loss: 0.2168
Epoch 1974/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2975 - disc_loss: 0.1596 - rmse: 0.7709 - val_loss: 0.1964
Epoch 1975/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0530 - disc_loss: 0.1476 - rmse: 0.2326 - val_loss: 0.1984
Epoch 1976/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.8519 - disc_loss: 0.1448 - rmse: 0.1958 - val_loss: 0.2017
Epoch 1977/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1372 - disc_loss: 0.1482 - rmse: 0.2002 - val_loss: 0.2115
Epoch 1978/2000
1/1 [=======

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [17]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)



(67656, 24)
tn


In [18]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (47359, 24) val_df.shape :  (13532, 24) test_df.shape: (6765, 24)


In [19]:
#fake_df.shape[0]/8760

In [20]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [21]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [22]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum/nitrogen/


In [23]:
rnn_epochs, rnn_in_setps

(15, 240)

## 모델 학습

In [34]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    #training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
5/5 [==============================] - 9s 2s/step - loss: 0.5773 - mean_absolute_error: 0.6012 - nse: 0.0334 - val_loss: 0.3019 - val_mean_absolute_error: 0.4158 - val_nse: -0.0176
Epoch 2/15
5/5 [==============================] - 9s 2s/step - loss: 0.3633 - mean_absolute_error: 0.4705 - nse: 0.3529 - val_loss: 0.2084 - val_mean_absolute_error: 0.3414 - val_nse: 0.2928
Epoch 3/15
5/5 [==============================] - 9s 2s/step - loss: 0.3054 - mean_absolute_error: 0.4099 - nse: 0.4832 - val_loss: 0.2026 - val_mean_absolute_error: 0.3423 - val_nse: 0.3163
Epoch 4/15
5/5 [==============================] - 9s 2s/step - loss: 0.2595 - mean_absolute_error: 0.3818 - nse: 0.5558 - val_loss: 0.1472 - val_mean_absolute_error: 0.2863 - val_nse: 0.5059
Epoch 5/15
5/5 [==============================] - 9s 2s/step - loss: 0.2340 - mean_absolute_error: 0.3592 - nse: 0.6127 - val_loss: 0.1483 - val_mean_absolute_error: 0.2863 - val_nse: 0.5027
Epoch 6/15
5/5 [============================

In [35]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/geum/models/geum/nitrogen/
year : 2013 ~ 2020
run :  [0]
target :  tn   5
length :  67656
train=7, val=1, test=2
all_nse :  [0.6451049]
all_pbias :  [0.01607754]
all_mae :  0.09971692
all_rmse :  0.13966757
all_Rs :  0.6462969990991496
all_R :  0.803926


In [36]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.6712532]
train_pbias :  [0.3685622]
train_mae :  0.09915647
train_rmse :  0.14082843
train_Rs :  0.6731876080521602
train_R :  0.8204801


In [37]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [0.5697838]
val_pbias :  [1.9096917]
val_mae :  0.117175154
val_rmse :  0.15119988
val_Rs :  0.5851585899597609
val_R :  0.7649566


In [38]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.5062138]
test_pbias :  [-1.3287781]
test_mae :  0.09262514
test_rmse :  0.12796444
test_Rs :  0.5180393575601592
test_R :  0.7197495


In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1

